# notMNIST letters visualization 

In this notebook, we'll apply some popular visualization techniques to visualize notMNIST letters using a GPU and the [RAPIDS](https://rapids.ai/) libraries (cudf, cuml).  This notebook is based on the scikit-learn embedding examples found [here](http://scikit-learn.org/stable/auto_examples/manifold/plot_lle_digits.html).

**Note that a GPU is required with this notebook.**

This version of the notebook has been tested with RAPIDS version 0.15.

First, the needed imports.

In [ ]:
%matplotlib inline

from time import time

import cudf
import numpy as np
import pandas as pd

import os
import urllib.request

from cuml import PCA, TSNE, UMAP
from cuml.random_projection import SparseRandomProjection
from cuml import __version__ as cuml_version

import sklearn
from sklearn import __version__ as sklearn_version

import matplotlib.pyplot as plt

print('Using cudf version:', cudf.__version__)
print('Using cuml version:', cuml_version)
print('Using sklearn version:', sklearn_version)

Then we load the notMNIST data. First time we need to download the data, which can take a while. The data is stored as Numpy arrays in host (CPU) memory.

In [ ]:
def load_not_mnist(directory, filename):
    filepath = os.path.join(directory, filename)
    if os.path.isfile(filepath):
        print('Not downloading, file already exists:', filepath)
    else:
        if not os.path.isdir(directory):
            os.mkdir(directory)
        url_base = 'https://a3s.fi/mldata/'
        url = url_base + filename
        print('Downloading {} to {}'.format(url, filepath))
        urllib.request.urlretrieve(url, filepath)
    return np.load(filepath)

In [ ]:
DATA_DIR = os.path.expanduser('~/data/notMNIST/')
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)
    
X = load_not_mnist(DATA_DIR, 'notMNIST_large_images.npy').reshape(-1, 28*28)
X = X.astype(np.float32)
y = load_not_mnist(DATA_DIR, 'notMNIST_large_labels.npy')

print()
print('notMNIST data loaded:',len(X))
print('X:', type(X), 'shape:', X.shape, X.dtype)
print('y:', type(y), 'shape:', y.shape, y.dtype)

Let's convert our data to a cuDF DataFrame in device (GPU) memory. 

In [ ]:
%%time

cu_X = cudf.DataFrame.from_pandas(pd.DataFrame(X))

print('cu_X:', type(cu_X), 'shape:', cu_X.shape)

Let's start by inspecting our data by drawing some samples:

In [ ]:
n_img_per_row = 32 # 32*32=1024
img = np.zeros((28 * n_img_per_row, 28 * n_img_per_row))

for i in range(n_img_per_row):
    ix = 28 * i
    for j in range(n_img_per_row):    
        iy = 28 * j
        img[ix:ix + 28, iy:iy + 28] = X[i * n_img_per_row + j,:].reshape(28,28)
img = np.max(img)-img

plt.figure(figsize=(9, 9))
plt.imshow(img, cmap='Greys')
plt.title('1024 first notMNIST letters')
ax=plt.axis('off')

Let's define a helper function to plot the different visualizations:

In [ ]:
def plot_embedding(X, y, title=None, time=None, color=True, save_as=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    y_int = [ord(yi)-ord('A') for yi in y]
    
    fig = plt.figure(figsize=(16,9))
    ax = fig.add_subplot(111)
    plt.axis('off')

    if color:
        colors = plt.cm.tab10(y_int)
        #colors = plt.cm.Set1([yi / 10. for yi in y_int])
        alpha = 0.2
    else:
        colors = 'k'
        alpha = 0.2
    
    s = plt.scatter(X[:, 0], X[:, 1], 0.5, color=colors, alpha=alpha)

    if color:
        x_lim, y_lim = ax.get_xlim(), ax.get_ylim()
        x_labs = x_lim[0] + (x_lim[1]-x_lim[0])*np.arange(10)/30
        y_labs = y_lim[0] + (y_lim[1]-y_lim[0])*0.1*np.zeros(10)
        labels=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
        for i in range(10):
            plt.text(x_labs[i], y_labs[i], labels[i], color=plt.cm.tab10(i),
                     fontdict={'weight': 'bold', 'size': 32})

    if title is not None:
        if t0 is not None:
            plt.title("%s (%.2fs)" % (title, time))
        else:
            plt.title(title)
            
    if save_as is not None:
        plt.savefig(save_as, bbox_inches='tight', dpi=150)

## 1. Random projection

A simple first visualization is a [random projection](http://scikit-learn.org/stable/modules/random_projection.html#random-projection) of the data into two dimensions.

In [ ]:
t0 = time()
rp = SparseRandomProjection(n_components=2, random_state=42)
X_projected = rp.fit_transform(cu_X).as_matrix()
t = time() - t0

In [ ]:
plot_embedding(X_projected, y, "Random projection", t, color=False)

Since we know the labels for our data, we can color the visulization accordingly.

In [ ]:
plot_embedding(X_projected, y, "Random projection", t)

## 2. PCA

[Principal component analysis](http://scikit-learn.org/stable/modules/decomposition.html#pca) (PCA) is a standard method to decompose a high-dimensional dataset in a set of successive orthogonal components that explain a maximum amount of the variance. Here we project the data into two first principal components. The components have the maximal possible variance under the orthogonality constraint.

In [ ]:
t0 = time()
pca = PCA(n_components=2)
X_pca = pca.fit_transform(cu_X).as_matrix()
t = time() - t0

In [ ]:
plot_embedding(X_pca, y, "PCA projection", t, color=False)

In [ ]:
plot_embedding(X_pca, y, "PCA projection", t)

## 4. t-SNE

[t-distributed Stochastic Neighbor Embedding](http://scikit-learn.org/stable/modules/manifold.html#t-sne) (t-SNE) is a relatively new and popular tool to visualize high-dimensional data.  t-SNE is particularly sensitive to local structure and can often reveal clusters in the data.

t-SNE has an important tuneable parameter called `perplexity`, that can have a large effect on the resulting visualization, depending on the data.  Typical values for perplexity are between 5 and 50.  

In [ ]:
t0 = time()
perplexity=30
tsne = TSNE(n_components=2, perplexity=perplexity)
X_tsne = tsne.fit_transform(cu_X).as_matrix()
t = time() - t0

In t-SNE visualizations, there are sometimes outliers that distract the visualization. We therefore remove `prc` % of smallest and largest data points on both dimensions.

In [ ]:
prc = 0.01
min_tsne = np.percentile(X_tsne, prc, axis=0)
max_tsne = np.percentile(X_tsne, 100-prc, axis=0)
range_ok = ((X_tsne[:,0]>min_tsne[0]) & 
            (X_tsne[:,0]<max_tsne[0]) &
            (X_tsne[:,1]>min_tsne[1]) & 
            (X_tsne[:,1]<max_tsne[1]))
            
X_tsne_filt = X_tsne[range_ok]
y_tsne_filt = y[range_ok]

plt.figure(figsize=(5,2))
plt.hist(X_tsne_filt[:, 0], 100, 
         range=(min_tsne[0], max_tsne[0]), alpha=0.7);
plt.hist(X_tsne_filt[:, 1], 100, 
         range=(min_tsne[1], max_tsne[1]), alpha=0.7);

In [ ]:
plot_embedding(X_tsne_filt, y_tsne_filt, 
               "t-SNE embedding with perplexity=%d" % perplexity,
t)

## 5. UMAP

[Uniform Manifold Approximation and Projection](https://umap-learn.readthedocs.io/en/latest/index.html) (UMAP) is another recently published technique for data visualization and dimensionality reduction based on manifold learning and topological data analysis.

The main hyperparameters of UMAP include `n_neighbors` and `min_dist`, which control the the size of the local neighborhood considered and how tightly the algorithm packs neighboring points together, respectively. The values of both hyperparameters have a significant impact on the resulting visualization. 

In [ ]:
t0 = time()
n_neighbors = 15
min_dist = 0.1
umapmodel = UMAP(n_neighbors=n_neighbors, min_dist=min_dist)
X_umap = umapmodel.fit_transform(cu_X).as_matrix()
t = time() - t0

In UMAP visualizations, there are sometimes outliers that distract the visualization. We therefore remove `prc` % smallest and largest data points on both dimensions.

In [ ]:
prc = 1.0
min_umap = np.percentile(X_umap, prc, axis=0)
max_umap = np.percentile(X_umap, 100-prc, axis=0)
range_ok = ((X_umap[:,0]>min_umap[0]) & 
            (X_umap[:,0]<max_umap[0]) &
            (X_umap[:,1]>min_umap[1]) & 
            (X_umap[:,1]<max_umap[1]))
            
X_umap_filt = X_umap[range_ok]
y_umap_filt = y[range_ok]

plt.figure(figsize=(5,2))
plt.hist(X_umap_filt[:, 0], 100, 
         range=(min_umap[0], max_umap[0]), alpha=0.7);
plt.hist(X_umap_filt[:, 1], 100, 
         range=(min_umap[1], max_umap[1]), alpha=0.7);

In [ ]:
plot_embedding(X_umap_filt, y_umap_filt,
               "UMAP projection with n_neighbors=%d, min_dist=%.2f" % (n_neighbors, 
                                                                       min_dist),
               t)

## 3D visualizations

In this section, we produce some 3D visualizations using [Plotly](https://plotly.com/python/).

In [ ]:
import plotly.express as px
from plotly.offline import plot

### PCA

In [ ]:
t0 = time()
pca3 = PCA(n_components=3)
X_pca3 = pca3.fit_transform(cu_X).as_matrix()
print('{:.2f} seconds elapsed'.format(time() - t0))

In [ ]:
fig = px.scatter_3d(
    X_pca3, x=0, y=1, z=2, color=y,
    title='PCA projection',
    labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
)
fig.update_traces(marker=dict(size=1))
plot(fig, filename = 'pca3-plot.html');

### UMAP

In [ ]:
t0 = time()
n_neighbors = 15
min_dist = 0.1
umapmodel3 = UMAP(n_components=3, 
                  n_neighbors=n_neighbors, min_dist=min_dist)
X_umap3 = umapmodel3.fit_transform(cu_X).as_matrix()
print('{:.2f} seconds elapsed'.format(time() - t0))

In [ ]:
prc = 1.0
min_umap3 = np.percentile(X_umap3, prc, axis=0)
max_umap3 = np.percentile(X_umap3, 100-prc, axis=0)
range_ok = ((X_umap3[:,0]>min_umap3[0]) & 
            (X_umap3[:,0]<max_umap3[0]) &
            (X_umap3[:,1]>min_umap3[1]) & 
            (X_umap3[:,1]<max_umap3[1]) &
            (X_umap3[:,2]>min_umap3[2]) & 
            (X_umap3[:,2]<max_umap3[2]))
            
X_umap3_filt = X_umap3[range_ok]
y_umap3_filt = y[range_ok]

In [ ]:
fig = px.scatter_3d(
    X_umap3_filt, x=0, y=1, z=2, color=y_umap3_filt,
    title="UMAP projection with n_neighbors=%d, min_dist=%.2f" % (n_neighbors, 
                                                                  min_dist),
    labels={'0': 'UMAP 1', '1': 'UMAP 2', '2': 'UMAP 3'}
)
fig.update_traces(marker=dict(size=1))
plot(fig, filename = 'umap3-plot.html');